# Mesh example
### Example of how to use the mesh functionality in JuAFEM using a Taylor-Hood element for incompressible elasticity

In [1]:
include("../src/JuAFEM.jl")
using JuAFEM

In [14]:
function main()
    # Set up a mesh
    xs = [0.0, 0.0] # Start coordinate
    xe = [2.0, 2.0] # End coordinate
    nel = [10,10] # Number of elements in each dir.
    
    # Generate a mesh object
    fe_m = FEMesh(Dim{2},JuAFEM.Triangle(),xs,xe,nel)
    
    # Set up two fields with different FE approximation order
    # Quadratic displacement
    u_fs = Lagrange{2, JuAFEM.Triangle, 2}()
    u_qr = get_gaussrule(Dim{2}, JuAFEM.Triangle(), 2)
    u_fe_v = FEValues(Float64, u_qr, u_fs)
    u_field = FEField(u_fs,2,"displacement")
    
    # Linear pressure
    p_fs = Lagrange{2, JuAFEM.Triangle, 1}()
    p_qr = get_gaussrule(Dim{2}, JuAFEM.Triangle(), 1)
    p_fe_v = FEValues(Float64, p_qr, p_fs)
    p_field = FEField(p_fs,1,"pressure")
    
    fields = [u_field,p_field]
    
    # Generate a FEDofs object
    fe_d = FEDofs(fe_m,fields)
    
    # Material parameters
    G = 10.0 # Shear modulus
    
    
    # Since this problem is linear we can just assemble the stiffness matrix and solve
    _K = start_assemble() # Assembler
    
    for el in 1:JuAFEM.get_number_of_elements(fe_m)
        return get_element_dofs(fe_d,1)
        # Get node coordinates for first field, and reinitiate its fe_values
        x_u = get_node_coordinates(fe_d,el,1)
        reinit!(get_fe_values(fe_d,1),x_u)
        
        # Get node coordinates for second field, and reinitiate its fe_values
        x_p = get_node_coordinates(fe_d,el,2)
        reinit!(get_fe_values(fe_d,2),x_p)
        
        S = [Tensor{2,2}(zeros(2,2)) for i in 1:n_basefunc_u]
        #C = ...
        for q_point in 1:length(JuAFEM.points(get_quadrule(u_fe_v)))
            I = Tensor{2,2}(eye(2))
            B_dev_sym = function_symmetric_gradient(u_fe_v) - 1/3 * trace(function_symmetric_gradient(u_fe_v)) * I
            
            for iu in 1:n_basefunc_u
                S[iu] += 2 * G * dcont(B_dev_sym,B_dev_sym)
            end
            
            for ip in 1:n_basefunc_p
                
            end
        end
        assemble(m,_K,ke)
    end
    K = end_assemble(_K)
    
    a,fb = solveq(K,f,bc)
    
    # Write to VTK
end

main (generic function with 1 method)

In [15]:
main()

9-element Array{Int64,1}:
  1
  2
  4
  5
 34
 35
  3
  6
 36